# ABI - versão inicial 2

(Descrição, ao finalizar)

## Parte 0: importação de bibliotecas

Usaremos as seguintes bibliotecas (TODO: adicionar links para a documentação):

- Numpy: para vetorização de dados;
- Pandas: para manipulação de base dados;
- NLTK: para ferramentas de NLP;
- Re: para processamento de regex;
- ast: pra trasnformar string em dicionário;
- Scikitlearn: para modelagem estatística e machine learning;
- TensorFlow: framework de redes neurais e deep learning;
- Keras: API de alto nível para o TensorFlow;
- Matplotlib: para plots e visualizações
- Seaborn: para plotagem estatística

### Importando de principais bibliotecas

In [ ]:
#processamento de dados
import numpy as np
import pandas as pd

#regex
import re

#nlp
import nltk
# nltk.download('stopwords')
# nltk.download('rslp')

#dataviz
import matplotlib.pyplot as plt

#random
import random
#pra termos resultados mais fixos

#pra tirar acentos
import unidecode

#pra transformar string em dict
import ast

## Parte 1: leitura de dados

### Lendo a base de perguntas

In [ ]:
#numero de perguntas pra cada intent
n = 100

df_q = pd.read_csv("base_treino_" + str(n) + ".csv")

#é bom dar uma misturada...
df_q = df_q.sample(frac=1, random_state = 42).reset_index(drop=True)
 
#essa é a base desbalanceada
df_desbalanc = df_q[~df_q["intent"].isin(df_q["intent"].value_counts()[df_q["intent"].value_counts() == n].index.tolist())]

#essa é a base balanceada (vou manter o mesmo nome)
df_q = df_q[df_q["intent"].isin(df_q["intent"].value_counts()[df_q["intent"].value_counts() == n].index.tolist())]
    
if (~df_q["intent"].value_counts().values == n).sum() == 0:
    print("Bases balanceada e desbalanceadas separadas com sucesso!")
    display(df_q.head())
else:
    print("Erro... Olha o código!")
    assert(False)

In [ ]:
df_q.info()

In [ ]:
perguntas = df_q.iloc[:,1]
print("\nTemos", len(perguntas), "perguntas na base.\nAs perguntas estão distribuídas em diferentes intents:\n")

intents = df_q.iloc[:,0]
print("Temos", len(intents.unique()), "intents na base, com as respectivas quantidades de perguntas",
                                      "(que idealmente devem ser balanceadas entre os intents):")
display(intents.value_counts())

## Parte 2: pré-processamento

### Criação do corpus a partir das perguntas da base de treino

Etapas de pré-processamento (todas são discutíveis):

- limpeza do texto: retiramos números e pontuação;
- deixamos todo o texto em minúsculas;
- aplicamos stemming. Temos duas opções de stemmers:
    - RSLP Stemmer: http://www.inf.ufrgs.br/~viviane/rslp/index.htm
    - Snowball (Porter2): https://snowballstem.org/
- retiramos as stopwords das perguntas

In [ ]:
#stopwords
stpwrds = nltk.corpus.stopwords.words('portuguese')

#essa é a função de pre-processamento
def pre_proc(pergunta, stpwrds=stpwrds):
    
    #vamos jogar fora tudo que não são letras minusculas e maiusculas (incluido acentuações)
    #vou manter também apenas os numeros 0,1,2, pra poder captar 110 e 220
    pergunta = re.sub("[^a-zA-ZáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ0-2]", " ", pergunta)

    #deixa tudo minuscula
    pergunta = pergunta.lower()

    #tonkeniza
    pergunta = pergunta.split()

    #stemmer SB
    stemmer = nltk.stem.SnowballStemmer('portuguese')
        
    #tira stopwprds. tora acentps e aplica o stemmer
    pergunta = [stemmer.stem(unidecode.unidecode(word)) for word in pergunta if word not in set(stpwrds)]

    #refaz a string
    pergunta = " ".join(pergunta)

    return pergunta

corpus = []

for item in perguntas:

    pergunta = pre_proc(item, stpwrds)

    #adiciona ao corpus
    corpus.append(pergunta)

In [ ]:
# print("\nComparação entre a pergunta original e a versão pré-processada da pergunta no corpus:\n")

# for i in range(len(perguntas)):
#     print(perguntas[i], "|", corpus[i])

### Bag of words

Pra começar, vamos fazer um pequeno estudo quanto ao vocabulário do Corpus

In [ ]:
#só pra ter uma ideia do vocabulário, vamos fazer uma lista de listas com o formato:
#vocabulario[i] = [palavra, numero_de_aparicoes_no_corpus]

vocabulario = []
for pergunta in corpus:
    for palavra in pergunta.split():
        #não queremos palavras de uma única letra (pode acontecer devido ao stemming...)
        if len(palavra) > 1:
            if palavra not in [x[0] for x in vocabulario]:
                vocabulario.append([palavra, 1])
            else:
                vocabulario[[x[0] for x in vocabulario].index(palavra)][1] += 1
            
print("\nO vocabulário é formado por", len(vocabulario), "palavras!")

#a partir do vocabulário, crio um dataframe com a contagem
vocab_count = pd.DataFrame({"palavra": [],
                           "count": []})

vocab_count["palavra"] = pd.Series(vocabulario).apply(lambda x: x[0])
vocab_count["count"] = pd.Series(vocabulario).apply(lambda x: x[1])
vocab_count = vocab_count.sort_values("count", ascending=False)

print("\nTemos a seguir as 10 mais comuns, com as respectivas contagens:")
display(vocab_count.head(10))

raras = vocab_count[vocab_count["count"] == 1]
prop_raras = raras.shape[0]/vocab_count.shape[0]
print("\nA quantidade de palavras com apenas uma aparição no corpus (palavras raras)\né de " + str(raras.shape[0]) +
     ", o que equivale a", str(round(100*(prop_raras), 2)) + "% da base!!")

__TODO: (maybe): pequena análise exploratória da frequencia relativa das palavras__

Para fazer o bag of words propriamente, vamos usar o CountVectorizer do sklearn:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#o max_feacutres imita o tamanho do vocabulario.. Não sei o quanto é interessante
# cv = CountVectorizer(max_features = 1500)
cv = CountVectorizer()

#features
X = cv.fit_transform(corpus).toarray()

#target
# y = intents_num
y = intents.values

#validação
if X.shape[1] == vocab_count.shape[0] and X.shape[0] == len(perguntas):
    print("As dimensões da matrix de features estão corretas!")
    print("\nBag of words feito com sucesso!")
else:
    print("As dimensões não batem! Reveja o código!")
    for item in [x[0] for x in vocabulario]:
        if item not in list(cv.vocabulary_.keys()):
            print(item)
    assert(False)

## Parte 3: modelagem preditiva

Testaremos agora diferentes modelos para a previsão dos intents com base nas perguntas

In [ ]:
#train-test split
from sklearn.model_selection import train_test_split

#avaliação de performanace
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)


print("Check de balanço do train-test split:")
display(pd.Series(y_train).value_counts())
display(pd.Series(y_test).value_counts())

In [ ]:
#############################################################################################
#############################################################################################
############################################################################################# 
######################################## MODELOS

#Modelos:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#instanciando os modelos
modelos = {"gnb": GaussianNB(),
           "mnb": MultinomialNB(),
           "logit": LogisticRegression(solver="lbfgs", multi_class="multinomial", random_state = 42),
           "svm": SVC(decision_function_shape='ovo', gamma="auto", kernel="linear", C=10, random_state = 42),
           "rf": RandomForestClassifier(n_estimators=300, max_depth=None, random_state = 42)}

#nesta lista eu vou colocar todos os modelos que eu fitar em cada modo, com as respectivas indicações
#formato da lista modelos_fitados:
#[[nome_do_modelo, modelos_fitados]*numero_de_modelos]
modelos_fitados = []

#nessa lista eu vou guardar apenas os melhores modelos de cada modo
#formato da lista melhores_modelos_fitados
#[[nome_do_melhor_modelo, weighted_f1_do_melhor_modelo, melhor_modelos_fitados]*1]
melhores_modelos_fitados = []

#inicializa o melhor f1 como zero
melhor_f1 = 0
for modelo in list(modelos.keys()):

    print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ | ~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~ | ~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

    print("Modelo:", modelo)
    print("\nParâmetros:", modelos[modelo])

    fit = modelos[modelo].fit(X_train, y_train)
    y_pred = fit.predict(X_test)

    print("\nAvaliação:\n")
    # print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    #salva o modelo fitado na forma modelos_fitados[i] = [nome_do_modelo, modelos_fitados]
    modelos_fitados.append([modelo, fit])

    #vendo qual é o melhnor modelo
    cr = classification_report(y_test, y_pred, output_dict=True)
    f1 = cr["weighted avg"]["f1-score"]

    if f1 > melhor_f1:
        melhor_f1 = f1
        melhor = modelo
        melhor_fit = fit

print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

print("\nModelo com maior weighted f1:", melhor, ", com weighted f1 de", melhor_f1)

print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")  

melhores_modelos_fitados.append([melhor, melhor_f1, melhor_fit])

## Rede Neural

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

random.seed(42)
np.random.seed(42)
import tensorflow as tf
tf.random.set_random_seed(42)

#cria modelo
def nn_model(optimizer="adam", alpha=0.01):

    ###################### ARQUITETURA DA REDE NEURAL ######################

    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(intents.unique()), activation='softmax'))

    ###################### OPTIMIZER ######################

    if optimizer == "adam":
        opt = keras.optimizers.Adam(lr=alpha)
    elif optimizer == "rmsprop":
        opt = keras.optimizers.RMSprop(lr=alpha)
    elif optimizer == "sgd":
        opt = keras.optimizers.SGD(lr=alpha, decay=1e-6, momentum=0.9, nesterov=True)

    ###################### COMPILAÇÃO DO MODELO ######################

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model
    
#dicionario de modelos
modelos_nn = {}

#lista de parâmetros pros diferentes modelos
#formato: [optimizer, alpha(lr), n_epochs, batch_size, nome_do_modelo]
parametros = [
                ["rmsprop", 0.001, 10, 10, "opt=rmsprop | lr=0.001 | n_epochs=10 | batch_size=10"],
                ["rmsprop", 0.001, 150, 10, "opt=rmsprop | lr=0.001 | n_epochs=100 | batch_size=10"],
            ]

for item in parametros:
    
    modelos_nn[item[-1]] = KerasClassifier(build_fn = nn_model, optimizer = item[0], alpha = item[1], epochs=item[2], batch_size=item[3], verbose=0)


#nesta lista eu vou colocar todos os modelos que eu fitar em cada modo, com as respectivas indicações
#formato da lista modelos_fitados:
#[[nome_do_modelo, modelos_fitados]*numero_de_modelos]
modelos_fitados_nn = []

#nessa lista eu vou guardar apenas os melhores modelos de cada modo
#formato da lista melhores_modelos_fitados
#[[nome_do_melhor_modelo, weighted_f1_do_melhor_modelo, melhor_modelos_fitados]*1]
melhores_modelos_fitados_nn = []

#inicializa o melhor f1 como zero
melhor_f1 = 0
for modelo in list(modelos_nn.keys()):

    print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ | ~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~ | ~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

    print("Modelo:", modelo)
    print("\nParâmetros:", modelos_nn[modelo])

    modelos_nn[modelo].fit(X_train, y_train)
    y_pred = modelos_nn[modelo].predict(X_test)

    print("\nAvaliação:\n")
    # print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))   

    #salva o modelo fitado na forma modelos_fitados[i] = [nome_do_modelo, modelos_fitados]
    modelos_fitados_nn.append([modelo, modelos_nn[modelo]])

    #vendo qual é o melhnor modelo
    cr = classification_report(y_test, y_pred, output_dict=True)
    f1 = cr["weighted avg"]["f1-score"]

    if f1 > melhor_f1:
        melhor_f1 = f1
        melhor = modelo
        melhor_fit = modelos_nn[modelo]

print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

print("\nModelo com maior weighted f1:", melhor, ", com weighted f1 de", melhor_f1)

print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")  

melhores_modelos_fitados_nn.append([melhor, melhor_f1, melhor_fit])

## Parte 4: ligação com a base de produtos

Agora vamos ler a base de produto e implementar a estrutura de comunicação do modelo com ela pra criarmos as respostas do bot

### alterações:
- colocar o parcelamento da forma de pagamento;
- em acessórios, se tiver mais de um acessório que acompanha, coloca em plural a chave do dict;
- adicionar produtos usados em "estado" 
- deixar as chaves do dict com acentos, etc
- cores: nao deixar a ultima ser uma que nao tá disponivel
- "desconto" --> "com possibilidade de desconto"; "cartao_credito" --> "cartão de crédito"
- nao deixa uma forma de pagamento nao disponivel como ultima tbm

In [ ]:
base_de_produtos = pd.read_csv("base_produto.csv")

In [ ]:
# essa resposta constrói a resposta final de acordo com cada intent.
# os argumentos são: 
# o dicionario de intents dic_intent do respectivo produto; 
# o dicionario de respostas iniciais respostas_inicio; 
# o intent(que é output do modelo)
def resposta_especificas(dic_intent, intent):
    
    #dicionario com a base das respostas
    respostas_inicio = {'Acessórios': 'Olá! O(s) seguinte(s) item(s) vem junto com o produto: ', 
                         'Bateria': 'Olá! A bateria do produto tem as seguintes características: ', 
                         'Capacidade': 'Olá! O produto tem as seguintes especificações de capacidade: ', 
                         'Cor': 'Olá! Temos as seguintes cores disponíveis: ', 
                         'Dimensão': 'Olá! As dimensões do produto são: ', 
                         'Disponibilidade': 'Olá! ',
                         'Entrega': 'Olá! ',
                         'Estado': 'Olá! ', 
                         'Garantia': 'Olá! O produto tem as seguintes garantias: ', 
                         'Meios de pagamento': 'Olá! Os meios de pagamento que aceitamos são: ', 
                         'Nota Fiscal': 'Olá! ', 
                         'Voltagem': 'Olá! '}
    
    #no caso do produto não ter informação cadastrada pro respectivo intent
    if dic_intent == {}:
        resposta = "Olá! Não tenho esta informação cadastrada. Te retornarei novamente mais tarde!"
    else:
        #pros intents que tem textinho cadastrado, a resposta é o texto direto, e fim
        if "texto" in list(dic_intent.keys()):
            resposta = respostas_inicio[intent] + dic_intent["texto"]
        else:
            resposta = respostas_inicio[intent]

            if intent == 'Acessórios':

                #pra ajustar o ponto final
                count = 1
                for item in list(dic_intent.keys()):

                    #se tem apenas uma key no dict, eu coloco o ponto final
                    #coloco tb o ponto final se for a ultima key do dict (usando a variavel count)
                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):
                        resposta = resposta + str(dic_intent[item]) + " " + item + ". "
                    #se tem mais de um acessório, eu os separo por ponto e virgula
                    else:
                        resposta = resposta + str(dic_intent[item]) + " " + item + "; "

                    count += 1

            elif intent == 'Bateria':

                #listagem
                resposta = resposta + "\n"
                count = 1
                for item in list(dic_intent.keys()):

                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):
                        resposta = resposta + "- " + item + ": " + str(dic_intent[item]) + "."
                    else:
                        resposta = resposta + "- " + item + ": " + str(dic_intent[item]) + ";\n"

                    count += 1

            elif intent == 'Capacidade':

                resposta = resposta + "\n"
                count = 1
                for item in list(dic_intent.keys()):

                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):

                        #pro caso de eu ter uma lista de opçoes disponiveis (como pra memoria ram)
                        if type(dic_intent[item]) == list:
                            resposta = resposta + "- " + item + ": Temos as opções: " + "; ".join(dic_intent[item]) + "."
                        else:
                            resposta = resposta + "- " + item + ": " + str(dic_intent[item]) + "."
                    else:
                        if type(dic_intent[item]) == list:
                            resposta = resposta + "- " + item + ": Temos as opções: " + "; ".join(dic_intent[item]) + ";\n"
                        else:
                            resposta = resposta + "- " + item + ": " + str(dic_intent[item]) + ";\n"

                    count += 1

            elif intent == 'Cor':

                resposta = resposta + "\n"
                count = 1
                for item in list(dic_intent.keys()):

                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):
                        #se nao tiver daquela cor disponível
                        if dic_intent[item] != 0:
                            resposta = resposta + "- " + item + " (" + str(dic_intent[item]) + " unidade(s))" + ". "
                    else:
                        if dic_intent[item] != 0:
                            resposta = resposta + "- " + item + " (" + str(dic_intent[item]) + " unidade(s))" + ";\n"

                    count += 1

            elif intent == 'Dimensão':

                resposta = resposta + "\n"
                count = 1
                for item in list(dic_intent.keys()):

                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):

                        #pro caso de eu ter uma lista de opçoes disponiveis (como pra memoria ram)
                        if type(dic_intent[item]) == list:
                            resposta = resposta + "- " + item + ": Temos as opções: " + "; ".join(dic_intent[item]) + "."
                        else:
                            resposta = resposta + "- " + item + ": " + str(dic_intent[item]) + "."
                    else:
                        if type(dic_intent[item]) == list:
                            resposta = resposta + "- " + item + ": Temos as opções: " + "; ".join(dic_intent[item]) + ";\n"
                        else:
                            resposta = resposta + "- " + item + ": " + str(dic_intent[item]) + ";\n"

                    count += 1

            elif intent == 'Disponibilidade':

                #nessa coluna só vai ter o atributo "estoque", com a quantidade em estoque
                if dic_intent["estoque"] == 0:
                    resposta = resposta + "Infelizmente, o produto não está mais disponível."
                else:
                    resposta = resposta + "O produto está disponível! Temos " + str(dic_intent["estoque"]) + " unidades em estoque."

            elif intent == 'Garantia':

                resposta = resposta + "\n"
                count = 1
                for item in list(dic_intent.keys()):

                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):
                        resposta = resposta + "- " + item.replace("_", " ") + ": " + str(dic_intent[item]) + "."
                    else:
                        resposta = resposta + "- " + item.replace("_", " ") + ": " + str(dic_intent[item]) + ";\n"

                    count += 1

            elif intent == 'Meios de pagamento':

                count = 1
                for item in list(dic_intent.keys()):

                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):
                        #listo apenas as formas de pagamento disponíveis
                        if dic_intent[item] == "sim":
                            resposta = resposta + item.replace("_", " ") + ". "
                    else:
                        if dic_intent[item] == "sim":
                            resposta = resposta + item.replace("_", " ") + "; "

                    count += 1

            elif intent == 'Nota Fiscal':

                #so tem esse atributo no dicionario de nota fiscal
                if dic_intent["nota"] == "sim":
                    resposta = resposta + "O produto será enviado juntamente da Nota Fiscal!"
                elif dic_intent["nota"] == "nao":
                    resposta = resposta + "Não emitimos Nota Fiscal."

            elif intent == 'Voltagem':

                resposta = resposta + "Temos "
                count = 1
                for item in list(dic_intent.keys()):

                    if len(list(dic_intent.keys())) == 1 or count == len(list(dic_intent.keys())):
                        if dic_intent[item] != 0:
                            resposta = resposta + str(dic_intent[item]) + " unidades de " + item + "."
                    else:
                        if dic_intent[item] != 0:
                            resposta = resposta + str(dic_intent[item]) + " unidades de " + item + "; "

                    count += 1

    return resposta

In [ ]:
#essa é a função que dá a resposta do bot.
# seus argumentos são:
# - a base de produtos "base_de_produtos" (que eu já tenho de ter lido)
# - intent, que é o output do modelo
# - produto, que é o produto ao qual a pergunta se refere. Isso entra a partir da GUI
#dentro desta tem uma que é ainda mais importante, e que eu defini acima, a "resposta_especificas()"

def respostas_bot(prod = base_de_produtos, intent = "", produto = ""):

    #lista dos produtos construida a partir da base de produtos
    lista_de_produtos = [list(ast.literal_eval(item).values())[0] for item in prod["Produto"].tolist()]

    #ve qual é o indice de linha do dataframe de produtos correspondentes ao produto a que se refere a pergunta
    indice_linha = lista_de_produtos.index(produto)

    #dicionario do respectivo produto e do respectivo intent
    dic_intent = ast.literal_eval(prod.loc[indice_linha, intent])
    
    #completa a resposta
    resposta_final = resposta_especificas(dic_intent, intent)
                 
    return resposta_final

## Parte 5: Obot

Vamos pegar o melhor modelo dos que foram treinados acima, e usar aqui pra simular o bot

In [ ]:
#tem que vir da gui
produto = 'Samsung Galaxy J2'

usar_nn = False
def chat(model, th1 = 2):
    
    print("Digite sua pergunta sobre o produto!")
    
    while True:
        
        pergunta = input("Pergunta: ")
        
        if pergunta.lower() == "sair":
            break
        
        #faz o pre-processing da pergunta
        pergunta = pre_proc(pergunta)
            
        #eu tenho que fazer o bag da pergunta usando o mesmo cv e corpus de antes, claro
        X = cv.transform([pergunta]).toarray()
        
        #essas são as probabilidades das classes
        class_probs = model.predict_proba(X)
        
        #essa é a chance random
        th2 = 1/len(model.classes_)
        
        
        if usar_nn:
            #aqui, eu pego quais são as classes que são mais probáveis
            #a forma como eu defino é: as classes que tem mais de 1 std de desvio padrao dentro de todas as probs
            #pode ser que seja só uma, claro
            #mas pode ser que seja mais de uma... se for esse o caso, eu dou output de duas classes
            class_max = class_probs[abs(class_probs - np.mean(class_probs)) > 1*np.std(class_probs)]
        else:
            #aqui, eu pego quais são as classes que são mais probáveis
            #a forma como eu defino é: classes que têm probabilidade maior do que th1*th2 da probabilidade máxima
            #essas são as classes que o algoritmo mais acha que são as corretas
            #pode ser que seja só uma, claro
            #mas pode ser que seja mais de uma... se for esse o caso, eu dou output de duas classes
            class_max = class_probs[class_probs > class_probs.max() - th1*th2]
        
        
        #a lista de respostas, que depois eu dou um join
        resposta = []
        nao_sei = True
        for prob in class_max:
            
            if prob > th1*th2:
                
                y_pred = model.classes_[class_probs.squeeze().tolist().index(prob)]
                
                #aqui a resposta é formulada
                resposta.append(respostas_bot(prod = base_de_produtos, intent = y_pred, produto = produto))

                nao_sei = False
                
        #trasnforma as respostas em strings, separadas por "\n"
        resposta = "\n".join(resposta)
        
        #mas, se não houver nenhuma resposta muito provável...
        if nao_sei:
            resposta = "Eu não entendi o que você quis dizer..." 
        
        print(resposta)
        
        print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~||~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

#aqui escolhemos qual modelo usar

if usar_nn:
    #lista com os modelos tradicionais e as redes neurais tb
    melhores_modelos_geral = melhores_modelos_fitados_nn + melhores_modelos_fitados
else:
    melhores_modelos_geral = melhores_modelos_fitados

#pega o indice do melhor modelo (o com maior weighted f1)
idx = np.array([x[1] for x in melhores_modelos_geral]).argmax()

#seleciona o melhor modelo
melhor_modelo = melhores_modelos_geral[idx][-1]
nome = melhores_modelos_geral[idx][0]

print("\n################################################")
print("################################################\n")
print("Modelo utilizado:", nome)
print("\n################################################")
print("################################################\n\n")

#chama a função de chat
chat(model=melhor_modelo)

## Passo 6: Analytcs

Aqui apenas gero o df de analytics pra mandar pro Wesley

In [ ]:
df_test = pd.read_csv("base_teste_perguntas.csv")

In [ ]:
def escolhe_modelo(usar_nn = False):
    if usar_nn:
        #lista com os modelos tradicionais e as redes neurais tb
        melhores_modelos_geral = melhores_modelos_fitados_nn + melhores_modelos_fitados
    else:
        melhores_modelos_geral = melhores_modelos_fitados

    #pega o indice do melhor modelo (o com maior weighted f1)
    idx = np.array([x[1] for x in melhores_modelos_geral]).argmax()

    #seleciona o melhor modelo
    melhor_modelo = melhores_modelos_geral[idx][-1]
    nome = melhores_modelos_geral[idx][0]

    model = melhor_modelo
    
    return model


def previsao(x, usar_nn):
    
    model = escolhe_modelo(usar_nn)

    question = pre_proc(x)
    X = cv.transform([question]).toarray()

    class_probs = model.predict_proba(X)
    th2 = 1/len(model.classes_)

    if usar_nn:
        class_max = class_probs[abs(class_probs - np.mean(class_probs)) > 1*np.std(class_probs)]
    else:
        class_max = class_probs[class_probs > class_probs.max() - th1*th2]

    resposta = []
    nao_sei = True
    for prob in class_max:
        if prob > th1*th2:
            y_pred = model.classes_[class_probs.squeeze().tolist().index(prob)]
            resposta.append(y_pred)
            nao_sei = False
            
    resposta = " | ".join(resposta)
            
    if nao_sei:
        resposta = "Não sei"

    return resposta
    
    
def analysis(row):

    real = row["intent"]
    previsto = row["previsto"]
    
    real = real.lower().split(" | ")
    previsto = previsto.lower().split(" | ")
    
    score = 0
    for item in previsto:
        if item in real:
            score += 1
            
    if len(real) > 1:
        if score >= len(real)-1:
            ans = "acertou"
        else:
            ans = "errou"
    else:
        if score == len(real):
            ans = "acertou"
        else:
            ans = "errou"
        
    return ans

print("\nCom redes neurais:")
df_com_nn = df_test.copy()
df_com_nn["previsto"] = df_com_nn["pergunta"].apply(lambda x: previsao(x, usar_nn=True))
df_com_nn["analise"] = df_com_nn.apply(lambda row: analysis(row), axis=1)
display(df_com_nn["analise"].value_counts(normalize=True))


print("\n########################################\n")

print("\nSem redes neurais:")
df_sem_nn = df_test.copy()
df_sem_nn["previsto"] = df_sem_nn["pergunta"].apply(lambda x: previsao(x, usar_nn=False))
df_sem_nn["analise"] = df_sem_nn.apply(lambda row: analysis(row), axis=1)
display(df_sem_nn["analise"].value_counts(normalize=True))

In [ ]:
df_com_nn.to_excel("base_teste_com_nn.xlsx")
df_sem_nn.to_excel("base_teste_sem_nn.xlsx")

__Frequência de intents__

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,10))

data = df_test["intent"].value_counts(normalize=True)

sns.barplot(x = data.index.tolist(), y = data.values.tolist())
plt.title("Frequência relativa de intents na base de teste")
plt.xlabel("Intents")
plt.xticks(rotation=60)
plt.ylabel("Frequência relativa")
plt.show()

In [ ]:
for item in df_sem_nn["intent"].unique().tolist():
    
    print("Intent:", item)
    
    print("\nDe todas as perguntas, as frequencias de acerto e erro são:")
    aux = df_sem_nn[df_sem_nn["intent"] == item]
    count = aux["analise"].value_counts(normalize=True)
    display(count)
    
    plt.subplots(1, 2, figsize=(10, 6))
    
    plt.subplot(1,2,1)
    g = sns.barplot(x = count.index.tolist(), y = count.values.tolist())
    plt.title("Frequência de\nacertos e erros\ndo intent " + item)
    plt.xlabel("Intents")
    plt.xticks(rotation=60)
    plt.ylabel("Frequência relativa")
    
    ax=g
    #annotate axis = seaborn axis
    for p in ax.patches:
                 ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha='center', va='center', fontsize=11, color='gray', xytext=(0, 10),
                     textcoords='offset points')
    _ = g.set_ylim(0, 1.2) #To make space for the annotations

    
    print("\nDe todas as perguntas QUE FORAM RESPONDIDAS (i.e., tirando as que ele respondeu 'Não sei', as frequencias de acerto e erro são:")
    aux = aux[aux["previsto"] != "Não sei"]
    count = aux["analise"].value_counts(normalize=True)
    display(count)
    
    
    plt.subplot(1,2,2)
    g=sns.barplot(x = count.index.tolist(), y = count.values.tolist())
    plt.title("Frequência de\nacertos e erros\ndo intent " + item + ",\nentre perguntas\nefetivamente respondidas")
    plt.xlabel("Intents")
    plt.xticks(rotation=60)
    plt.ylabel("Frequência relativa")
    
    ax=g
    #annotate axis = seaborn axis
    for p in ax.patches:
                 ax.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                     ha='center', va='center', fontsize=11, color='gray', xytext=(0, 10),
                     textcoords='offset points')
    _ = g.set_ylim(0, 1.2) #To make space for the annotations
    
    plt.show()
    
    
    plt.tight_layout()
    
    print("\n###############################################")
    print("###############################################\n")

## Passo 7: GUI

In [ ]:
lista_de_produtos

In [ ]:
import tkinter as tk

def coletar_produto(produto):
    
    global product_global
    product_global = produto
    
    foto = tk.PhotoImage(file="./imagens/"+produto+".png")
    label_imagem = tk.Label(frame_imagem, image=foto)
    label_imagem.image = foto
    label_imagem.place(relheight=1, relwidth=1)
    
    
################################################

def coletar_pergunta(question):
    
    ans = bot(th1=2, usar_nn=False, pergunta=question, produto=product_global) 
    
    texto_resposta = tk.Label(frame_resposta, bg='white', text=ans, anchor='nw', font=("Courier", 10), justify="left")
    texto_resposta.place(relheight=1, relwidth=1)

    return pergunta


################################################


def escolhe_modelo(usar_nn = False):
    if usar_nn:
        #lista com os modelos tradicionais e as redes neurais tb
        melhores_modelos_geral = melhores_modelos_fitados_nn + melhores_modelos_fitados
    else:
        melhores_modelos_geral = melhores_modelos_fitados

    #pega o indice do melhor modelo (o com maior weighted f1)
    idx = np.array([x[1] for x in melhores_modelos_geral]).argmax()

    #seleciona o melhor modelo
    melhor_modelo = melhores_modelos_geral[idx][-1]
    nome = melhores_modelos_geral[idx][0]

    model = melhor_modelo
    
    return model


################################################

def bot(th1=2, usar_nn=False, pergunta="", produto=""):

    model = escolhe_modelo(usar_nn)
    
    pergunta_pp = pre_proc(pergunta)

    X = cv.transform([pergunta_pp]).toarray()

    class_probs = model.predict_proba(X)

    th2 = 1/len(model.classes_)

    if usar_nn:
        class_max = class_probs[abs(class_probs - np.mean(class_probs)) > 1*np.std(class_probs)]
    else:
        class_max = class_probs[class_probs > class_probs.max() - th1*th2]

    resposta = []
    nao_sei = True
    for prob in class_max:
        if prob > th1*th2:
            y_pred = model.classes_[class_probs.squeeze().tolist().index(prob)]
            resposta.append(respostas_bot(prod = base_de_produtos, intent = y_pred, produto = produto))
            nao_sei = False
    resposta = "\n".join(resposta)

    if nao_sei:
        resposta = "Não entendi a pergunta. Vou direcioná-la ao vendedor, e ele te responderá o mais rápido possível!" 
        
    return(resposta)


################################################3


# configuração inicial

janela_principal = tk.Tk() # single window
janela_principal.title('www.marketplace.com.br')
canvas = tk.Canvas(janela_principal, height=700, width=1000) # tamanho da janela ao abrir
canvas.pack()

# cabeçalhos e bordas

frame_cabeçalho = tk.Frame(janela_principal, bg='#ffff33')
frame_cabeçalho.place(relwidth=1, relheight=0.2)
frame_produto = tk.Frame(janela_principal, bg='#cbcbb3', bd=1)
frame_produto.place(relwidth=0.3, relheight=0.05, relx=0.1, rely=0.275)
frame_imagem = tk.Frame(janela_principal)
frame_imagem.place(relwidth=0.3, relheight=0.22, relx=0.1, rely=0.35)


frame_pergunta = tk.Frame(janela_principal, bg='#cbcbb3', bd=1)
frame_pergunta.place(relwidth=0.6, relheight=0.1, relx=0.1, rely=0.65)
frame_resposta = tk.Frame(janela_principal, bg='#cbcbb3', bd=1)
frame_resposta.place(relwidth=0.6, relheight=0.1, relx=0.1, rely=0.85)

# botões

botao_produto = tk.Button(janela_principal, text='OK', bg='#3385ff', fg='white', command=lambda: coletar_produto(entrada_produto.get()))
botao_produto.place(relheight=0.05, relwidth=0.05, relx=0.41, rely=0.275)
botao_pergunta = tk.Button(janela_principal, text='Enviar Pergunta', bg='#3385ff', fg='white', command=lambda: coletar_pergunta(entrada_pergunta.get()))
botao_pergunta.place(relheight=0.05, relwidth=0.1, rely=0.675, relx=0.723)

# labels

label_produto = tk.Label(janela_principal, text='Produto', font=40, fg='blue', anchor='w')
label_produto.place(relheight=0.05, width=100, relx=0.1, rely=0.22)
label_pergunta = tk.Label(janela_principal, text='Pergunte ao vendedor', font=40, fg='blue', anchor='w')
label_pergunta.place(relheight=0.05, width=200, relx=0.1, rely=0.595)


label_resposta = tk.Label(janela_principal, text='Resposta', font=40, fg='blue', anchor='w')
label_resposta.place(relheight=0.05, width=200, relx=0.1, rely=0.795)


# entries

entrada_produto = tk.Entry(frame_produto, font=40, bg='white')
entrada_produto.place(relheight=1, relwidth=1)
entrada_pergunta = tk.Entry(frame_pergunta, font=100, bg='white')
entrada_pergunta.place(relheight=1, relwidth=1)


texto_resposta = tk.Label(frame_resposta, bg='white', text="", anchor='nw', font=("Courier", 10))
texto_resposta.place(relheight=1, relwidth=1)


janela_principal.mainloop()